In [22]:
import pandas as pd
import numpy as np
from datetime import datetime, time, timedelta
import requests

In [23]:
df = pd.read_csv("data/bangkok_traffy.csv")
df.describe()

,star,count_reopen
count,274097.000000,787026.000000
mean,3.872801,0.123485
std,1.471863,0.823554
min,1.000000,0.000000
25%,3.000000,0.000000
50%,5.000000,0.000000
75%,5.000000,0.000000
max,5.000000,100.000000


In [24]:
df.sample(5)

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
583935,ZEKTZB,"{เสียงรบกวน,ถนน,ร้องเรียน}","เขตป้อมปราบศัตรูพ่าย,ฝ่ายสิ่งแวดล้อมฯ เขตป้อมป...",ศูนย์เรื่องราวร้องทุกข์ ได้รับการประสานผ่านระบ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.51306,13.75824",สำนักงานเขตป้อมปราบศัตรูพ่าย ถนน ศุภมิตร แขวงว...,วัดโสมนัส,ป้อมปราบศัตรูพ่าย,จังหวัดกรุงเทพมหานคร,2024-05-30 03:20:29.413195+00,เสร็จสิ้น,NaN,0,2024-05-30 09:11:36.229146+00
635000,2024-DPC9TN,"{ความปลอดภัย,ร้องเรียน}","เขตจอมทอง,ฝ่ายโยธา เขตจอมทอง",ประชาชนร้องเรียนมาว่ามีกระจกมองทางโค้งชำรุด\nซ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.46990,13.70196",16/15 ซ. วุฒากาศ 5 แขวงบางค้อ เขตจอมทอง กรุงเท...,บางค้อ,จอมทอง,กรุงเทพมหานคร,2024-07-24 03:35:27.841699+00,กำลังดำเนินการ,NaN,0,2024-11-08 02:44:28.833726+00
229926,FL4ZHJ,"{PM2.5,ถนน}","สกส.กรร.สสสภ.,เขตลาดกระบัง,การประปานครหลวง กปน...",*กรุณาอ่านข้อมูลบรรยายประกอบการปักหมุด\nปัญหา ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.78329,13.72126",ลาดกระบัง 52 แยก 2 ลาดกระบัง ลาดกระบัง ลาดกระบ...,ลาดกระบัง,ลาดกระบัง,จังหวัดกรุงเทพมหานคร,2023-03-05 05:36:54.745302+00,เสร็จสิ้น,NaN,0,2023-05-02 07:19:51.985455+00
45482,2022-N3F822,{ท่อระบายน้ำ},"เขตจตุจักร,สำนักการระบายน้ำ กทม.,สำนักการโยธา ...",ฝาท่อเป็นรู ปากซอยลาดพร้าว 36,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.57795,13.80345",ก่อนแยกภาวนา แขวง จันทรเกษม เขตจตุจักร กรุงเทพ...,จันทรเกษม,จตุจักร,กรุงเทพมหานคร,2022-06-22 23:55:12.290206+00,เสร็จสิ้น,NaN,0,2023-02-15 04:38:01.07661+00
347687,2023-FA6PG4,"{แสงสว่าง,ทางเท้า}","เขตจตุจักร,สำนักการโยธา กทม.,สำนักงานก่อสร้างแ...",หน้าทางเดินจาก 7-11 ไปถึงชาดพร่าวซอย6 มืดมากค่ะ,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.56289,13.81165",158/18 ถ. ลาดพร้าว แขวงจอมพล เขตจตุจักร กรุงเท...,จอมพล,จตุจักร,กรุงเทพมหานคร,2023-08-13 05:00:18.663502+00,เสร็จสิ้น,5.0,0,2025-01-09 17:16:32.273715+00


In [25]:
finished_df = df[df['state'] == 'เสร็จสิ้น']

In [26]:
filtered_df = finished_df.dropna(axis=0, how='any', subset=['timestamp', 'last_activity'])

In [27]:
def toDate(serie):
    return pd.to_datetime(serie, format='ISO8601').dt.tz_localize(None)

filtered_df['timestamp'] = toDate(filtered_df['timestamp'])
filtered_df['last_activity'] = toDate(filtered_df['last_activity'])

filtered_df['duration'] = (filtered_df['last_activity'] - filtered_df['timestamp']).dt.total_seconds() // 60
filtered_df.sample(5)

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity,duration
321947,2023-4MFG9Q,{จราจร},"เขตสาทร,สน.บางโพงพาง,สำนักงานตำรวจแห่งชาติ (Ro...",การจราจร,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.52929,13.70441",9 11 ถ. สาธุประดิษฐ์ แขวง ช่องนนทรี เขต ยานนาว...,ทุ่งวัดดอน,สาทร,กรุงเทพมหานคร,2023-07-12 11:12:07.725805,เสร็จสิ้น,4.0,0,2023-12-19 18:25:23.404522,230833.0
513666,2024-MFLHDZ,"{ความปลอดภัย,ท่อระบายน้ำ}","แขวงทางหลวงธนบุรี กรมทางหลวง,เขตตลิ่งชัน,กรมทา...",ฝาท่อชำรุด ทำให้จักรยานล้ม แขนเจ็บ ควร แก้ไขโด...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.42585,13.78101",99/5 ถ. บรมราชชนนี แขวงฉิมพลี เขตตลิ่งชัน กรุง...,ฉิมพลี,ตลิ่งชัน,กรุงเทพมหานคร,2024-02-28 12:13:18.933131,เสร็จสิ้น,5.0,0,2024-03-14 02:44:15.176312,21030.0
492387,2024-8GCETD,{PM2.5},"เขตบางกะปิ,สำนักสิ่งแวดล้อม กทม.,ฝ่ายโยธา เขตบ...",ตึกสนามมวยก่อสร้างยังไม่มีติดตั้งผ้าใบกันฝุ่น ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.62558,13.75948",ซอย รามคำแหง 24/1 ถนน รามคำแหง แขวง หัวหมาก เข...,หัวหมาก,บางกะปิ,กรุงเทพมหานคร,2024-02-01 14:07:52.614333,เสร็จสิ้น,NaN,0,2024-02-02 06:48:31.766257,1000.0
748510,2024-A3LQ96,{ถนน},"เขตดินแดง,ฝ่ายเทศกิจ เขตดินแดง",มีการจอดรถมอเตอร์ไซค์กินพื้นที่ออกมาบริเวณด้าน...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.56546,13.76239",เลขที่ 322 18 ซอย รัชดาภิเษก 3 แขวงดินแดง เขตด...,ดินแดง,ดินแดง,กรุงเทพมหานคร,2024-11-29 14:39:54.162663,เสร็จสิ้น,NaN,0,2024-12-10 08:44:49.903872,15484.0
642029,2024-8VYCTL,{แสงสว่าง},"เขตห้วยขวาง,สำนักการโยธา กทม.,ศูนย์เครื่องมือก...",ไฟถนนดับ,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.59257,13.74832",PHXV+825 แขวงบางกะปิ เขตห้วยขวาง กรุงเทพมหานคร...,บางกะปิ,ห้วยขวาง,กรุงเทพมหานคร,2024-08-01 12:10:44.829191,เสร็จสิ้น,5.0,0,2024-09-03 09:10:49.046496,47340.0


In [28]:
filtered_df['type'].fillna('{}', inplace=True)
filtered_df['type'].isna().sum()

/tmp/ipykernel_11168/4244501731.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_df['type'].fillna('{}', inplace=True)


np.int64(0)

In [29]:
types = {}

def count_type(row):
    val = row['type']
    print(val)
    for t in val[1:-1].split(','):
        types[t] += 1

In [30]:
for data in filtered_df['type']:
    # print(data)
    for t in data[1:-1].split(','):
        if t not in types:
            types[t] = 0
        types[t] += 1
# for key in sorted(types, reverse=True, key=lambda x: types[x]):
#     print(f'{key} - {types[key]}')
print(set(list(types)))

{'', 'คลอง', 'ป้ายจราจร', 'จราจร', 'คนจรจัด', 'ร้องเรียน', 'ทางเท้า', 'ป้าย', 'น้ำท่วม', 'PM2.5', 'สอบถาม', 'การเดินทาง', 'ความสะอาด', 'เสียงรบกวน', 'ต้นไม้', 'สายไฟ', 'ถนน', 'ห้องน้ำ', 'สัตว์จรจัด', 'แสงสว่าง', 'เสนอแนะ', 'ความปลอดภัย', 'กีดขวาง', 'ท่อระบายน้ำ', 'สะพาน'}


In [31]:
# columns = ['type', 'organization', 'coords', 'timestamp', 'duration']

# main_df = filtered_df[columns]

In [32]:
# Define allowed categories
target_types = {
 '',
 'ป้าย',
 'ความสะอาด',
 'แสงสว่าง',
 'สอบถาม',
 'ร้องเรียน',
 'การเดินทาง',
 'จราจร',
 'ท่อระบายน้ำ',
 'สะพาน',
 'เสียงรบกวน',
 'ต้นไม้',
 'คนจรจัด',
 'คลอง',
 'ถนน',
 'เสนอแนะ',
 'กีดขวาง',
 'สายไฟ',
 'PM2.5',
 'น้ำท่วม',
 'ทางเท้า',
 'สัตว์จรจัด',
 'ความปลอดภัย',
 'ห้องน้ำ',
 'ป้ายจราจร'
}

# Function to convert string to list
def parse_categories(s):
    # Remove curly braces and split
    return s.strip('{}').split(',')

# Parse the string column
filtered_df['type_list'] = filtered_df['type'].apply(parse_categories)

# Create binary flags
for cat in target_types:
    filtered_df[cat] = filtered_df['type_list'].apply(lambda lst: int(cat in lst))

# Create 'Others' column
filtered_df['Others'] = filtered_df['type_list'].apply(
    lambda lst: int(any(c not in target_types for c in lst))
)

# Drop intermediate column if you want
filtered_df.drop(columns='type_list', inplace=True)

filtered_df[['type'] + list(target_types)].sample(5)

,type,,คลอง,ป้ายจราจร,จราจร,คนจรจัด,ต้นไม้,สายไฟ,ร้องเรียน,ถนน,...,PM2.5,เสนอแนะ,ความปลอดภัย,สอบถาม,การเดินทาง,ความสะอาด,เสียงรบกวน,กีดขวาง,ท่อระบายน้ำ,สะพาน
146941,{},1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
315512,{},1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
158877,{น้ำท่วม},0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
460130,"{แสงสว่าง,ถนน}",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
649716,{ถนน},0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# Define working hours
start_time = time(9, 30)
end_time = time(15, 30)

def minutes_to_next_working_hour(dt):
    weekday = dt.weekday()  # 0 = Monday, ..., 6 = Sunday
    current_time = dt.time()

    # If it's a weekend or after working hours, find next weekday at 9:30
    if weekday >= 5 or current_time >= end_time:
        # Add days until next weekday (Monday–Friday)
        days_ahead = 1
        while (dt + timedelta(days=days_ahead)).weekday() >= 5:
            days_ahead += 1
        next_working_start = datetime.combine((dt + timedelta(days=days_ahead)).date(), start_time)
        return int((next_working_start - dt).total_seconds() // 60)

    # Before work hours today
    elif current_time < start_time:
        today_start = datetime.combine(dt.date(), start_time)
        return int((today_start - dt).total_seconds() // 60)

    # During working hours
    else:
        return 0

# Apply function to timestamp column
filtered_df['until_working_time'] = filtered_df['timestamp'].apply(minutes_to_next_working_hour)

filtered_df[['timestamp', 'until_working_time']].sample(5)

,timestamp,until_working_time
114724,2022-08-10 13:53:07.032822,0
712575,2024-10-20 01:56:34.996383,1893
462006,2023-12-24 08:20:39.046016,1509
545797,2024-04-10 11:05:46.728602,0
140156,2022-09-10 03:25:22.048774,3244


# Org's info

In [34]:
limit = 100
org_key = 'bangkok'
url = f'https://publicapi.traffy.in.th/premium-org-fondue/statistic-rank/top-rank-avg-star?org_key={org_key}&limit={limit}'

resp = requests.get(url)
data = resp.json()
orgs = data['results']

pd.DataFrame(orgs).to_csv('data/org.csv')

In [59]:
orgs = pd.read_csv('data/org.csv')
orgs.sample(5)
orgs.drop(columns=['Unnamed: 0'], inplace=True)
# orgs.reset_index(inplace=True)
orgs.set_index('fonduegroup_name', inplace=True)
orgs.sample(5)

,rank,timestamp,fonduegroup_id,post_count_star,post_count_star_stat,avg_star,post_count_all,post_count_finish,post_finish_percentage,avg_duration_minutes_inprogress,avg_duration_minutes_finished
fonduegroup_name,,,,,,,,,,,
เขตหนองแขม,26,NaN,18052,3758,"{'star_1': 573, 'star_2': 147, 'star_3': 295, ...",3.96,12372,11236,90.82,8480,57869
เขตตลิ่งชัน,28,NaN,18016,4361,"{'star_1': 662, 'star_2': 164, 'star_3': 353, ...",3.95,15014,11194,74.56,15834,66577
สำนักสิ่งแวดล้อม กทม.,57,NaN,18129,9173,"{'star_1': 1749, 'star_2': 497, 'star_3': 885,...",3.72,24309,21640,89.02,8405,78992
เขตคลองเตย,56,NaN,18055,9727,"{'star_1': 1852, 'star_2': 486, 'star_3': 973,...",3.72,31160,24567,78.84,27273,92130
เขตสัมพันธวงศ์,46,NaN,18049,2219,"{'star_1': 413, 'star_2': 110, 'star_3': 183, ...",3.80,6060,5052,83.37,6778,45202


In [101]:
# Define target columns
target_cols = ['avg_star', 'post_finish_percentage', 'avg_duration_minutes_finished']

# Convert organization string to list
filtered_df['orgs_list'] = filtered_df['organization'].str.strip().str.split(',')

# Create filtered_df with NaNs in target columns
filtered_df[target_cols] = np.nan

# Explode orgs_list to merge with orgs
exploded = filtered_df[['orgs_list']].explode('orgs_list').reset_index()
exploded['orgs_list'] = exploded['orgs_list'].str.strip()

# Join with orgs DataFrame
merged = exploded.merge(orgs[target_cols], left_on='orgs_list', right_index=True, how='left')

# Aggregate mean values for each original row
means = merged.groupby('index')[target_cols].mean()

# Assign the means back to filtered_df
filtered_df.loc[means.index, target_cols] = means

# Result
filtered_df[['orgs_list'] + target_cols]


,orgs_list,avg_star,post_finish_percentage,avg_duration_minutes_finished
0,[เขตบางซื่อ],4.08,84.940,85415.0
1,"[เขตประเวศ, ฝ่ายโยธา เขตประเวศ]",3.82,84.320,76399.0
2,[เขตสาทร],3.86,84.510,54852.0
3,"[เขตบางซื่อ, ฝ่ายโยธา เขตบางซื่อ]",4.08,84.940,85415.0
4,"[เขตลาดพร้าว, ฝ่ายโยธา เขตลาดพร้าว]",4.09,85.420,50067.0
...,...,...,...,...
786856,"[เขตพญาไท, สำนักการจราจรและขนส่ง กรุงเทพมหานคร...",3.92,73.165,138599.0
786892,"[เขตบางพลัด, สน.บวรมงคล]",3.59,72.540,92196.0
786893,"[เขตบางเขน, สน.บางเขน, สำนักงานตำรวจแห่งชาติ (...",4.00,86.020,65512.0
786944,"[เขตบางกอกน้อย, ฝ่ายเทศกิจ เขตบางกอกน้อย]",3.87,84.180,72567.0


In [102]:
filtered_df.to_csv('data/clean.csv')